# CNN_SignMNIST

>### Load modules

In [1]:
import cv2
import os
import re
import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
import itertools
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.utils import to_categorical
from collections import defaultdict

print("NumPy Version :{}".format(np.__version__))
print("TensorFlow Version :{}".format(tf.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

NumPy Version :1.25.2
TensorFlow Version :2.15.0
Matplotlib Version :3.7.1


> ### Load Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')
print('g-drive mounted.')

Mounted at /content/drive
g-drive mounted.


In [3]:
files_path = '/content/PISC_relatoin_age_gender/'
if(os.path.isdir('/content/PISC_relatoin_age_gender/') == False):
  !unzip -q '/content/drive/MyDrive/files/PISC_relatoin_age_gender.zip'

In [4]:
%%time
IMG_SIZE = 224

dataset = defaultdict(lambda: defaultdict(dict))
filename_pattern = re.compile(r'image(\d+)_id(\d+)_([a-zA-Z]+)_(\w+)_(\w+).jpg')

for filename in tqdm(os.listdir(files_path), desc="Processing images", unit="image"):
    try:
        # Read and preprocess image
        img = cv2.imread(files_path+filename, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        match = filename_pattern.match(filename)
        if match:
            image_id, person_id, relationship, age, gender = match.groups()
            dataset[image_id][person_id] = {'image':img,'relationship': relationship, 'age': age, 'gender': gender}

    except Exception as e:
        print(f"Error processing {filename}: {e}")

input_data = []
output_data = []
relationship_table = {"friends":0,"family":1,"couple":2}
age_table = {"Infant":0,"Child":1,"YoungAdult":2,"MiddleAge":3,"Senior":4}

gender_table = {"male":0,"female":1}

for image_id, persons in dataset.items():
    person_ids = list(persons.keys())
    for person1_id, person2_id in itertools.combinations(person_ids, 2):
        person1 = persons[person1_id]
        person2 = persons[person2_id]

        age1 = age_table[person1['age']]
        age2 = age_table[person2['age']]
        gender1 = gender_table[person1['gender']]
        gender2 = gender_table[person2['gender']]
        relationship = relationship_table[person1['relationship']]

        input_data.append([
            person1['image'], age1, gender1,
            person2['image'], age2, gender2
        ])
        input_data.append([
            person2['image'], age2, gender2,
            person1['image'], age1, gender1
        ])
        output_data.append(relationship)
        output_data.append(relationship)


input_data = np.array(input_data, dtype=object)
output_data = to_categorical(output_data, num_classes=3)
print(len(input_data))
# Shuffle and split data
train_data, test_data, train_labels, test_labels = train_test_split(input_data, output_data, test_size=0.2, random_state=0)

def prepare_data(data):
    images1 = np.stack(data[:, 0])
    ages_genders1 = data[:, 1:3].astype('float32')
    images2 = np.stack(data[:, 3])
    ages_genders2 = data[:, 4:6].astype('float32')
    metadata = np.hstack((ages_genders1, ages_genders2))
    return [images1, images2, metadata]

train_data_prepared = prepare_data(train_data)
test_data_prepared = prepare_data(test_data)


Processing images: 100%|██████████| 716/716 [00:00<00:00, 986.21image/s] 


824
CPU times: user 652 ms, sys: 133 ms, total: 785 ms
Wall time: 843 ms


In [5]:
image_input1 = Input(shape=(224, 224, 3))
image_input2 = Input(shape=(224, 224, 3))
metadata_input = Input(shape=(4,))

# CNN for image 1
x1 = Conv2D(16, (3, 3), activation='relu')(image_input1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((4, 4))(x1)
x1 = Conv2D(16, (3, 3), activation='relu')(x1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((4, 4))(x1)
x1 = Flatten()(x1)
x1 = Dense(4, activation='relu')(x1)


# CNN for image 2
x2 = Conv2D(16, (3, 3), activation='relu')(image_input2)
x2 = Dropout(0.2)(x2)
x2 = MaxPooling2D((4, 4))(x2)
x2 = Conv2D(16, (3, 3), activation='relu')(x2)
x2 = Dropout(0.2)(x2)
x2 = MaxPooling2D((4, 4))(x2)
x2 = Flatten()(x2)
x2 = Dense(4, activation='relu')(x2)

# Concatenate image features and metadata
concatenated = Concatenate()([x1, x2, metadata_input])
dense = Dense(256, activation='relu')(concatenated)
output = Dense(3, activation='softmax')(dense)

model = Model(inputs=[image_input1, image_input2, metadata_input], outputs=output)
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 222, 222, 16)         448       ['input_1[0][0]']             
                                                                                                  
 conv2d_2 (Conv2D)           (None, 222, 222, 16)         448       ['input_2[0][0]']             
                                                                                              

In [8]:
savedModelName = 'relationship_prediction_model_v2.h5'
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(savedModelName,
                                    save_best_only=True)
]

In [7]:
model.fit(train_data_prepared, train_labels, epochs=50, batch_size=32,
          validation_data=(test_data_prepared, test_labels))

Epoch 1/50
21/21 [==============================] - 10s 94ms/step - loss: 2.0459 - accuracy: 0.4143 - val_loss: 1.0774 - val_accuracy: 0.3697
Epoch 2/50
21/21 [==============================] - 1s 49ms/step - loss: 1.0414 - accuracy: 0.5008 - val_loss: 0.9846 - val_accuracy: 0.5576
Epoch 3/50
21/21 [==============================] - 1s 44ms/step - loss: 0.9948 - accuracy: 0.6070 - val_loss: 0.9435 - val_accuracy: 0.6545
Epoch 4/50
21/21 [==============================] - 1s 45ms/step - loss: 0.9570 - accuracy: 0.5812 - val_loss: 0.9114 - val_accuracy: 0.6667
Epoch 5/50
21/21 [==============================] - 1s 49ms/step - loss: 0.9255 - accuracy: 0.5933 - val_loss: 0.8774 - val_accuracy: 0.6727
Epoch 6/50
21/21 [==============================] - 1s 54ms/step - loss: 0.8923 - accuracy: 0.5933 - val_loss: 0.8425 - val_accuracy: 0.6667
Epoch 7/50
21/21 [==============================] - 1s 53ms/step - loss: 0.8645 - accuracy: 0.5979 - val_loss: 0.8188 - val_accuracy: 0.6727
Epoch 8/50
2

In [9]:
model.save("/content/drive/MyDrive/files/save/"+savedModelName)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
save_dir = '/content/drive/MyDrive/files/save/'
model_best = tf.keras.models.load_model("/content/drive/MyDrive/files/save/"+savedModelName)
converter = tf.lite.TFLiteConverter.from_keras_model(model_best)
tflite_model = converter.convert()
tfliteFileName = 'relationship_prediction_model_v2.tflite'
open(save_dir + tfliteFileName, 'ab').write(tflite_model)

131076

In [ ]:
pred = model.predict(test_data_prepared)

id = 2
for id in range(len(pred)):
  rel = ["friend","family",'couple']
  # print(rel[np.argmax(pred[id])])
  # print(rel[np.argmax(test_labels[id])])
  # if np.argmax(pred[id]) != np.argmax(test_labels[id]):

  if np.argmax(pred[id]) == 2:
    # print(pred[id])
    print("pred:",rel[np.argmax(pred[id])])
    print("lable:",rel[np.argmax(test_labels[id])])
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(test_data_prepared[0][id])
    plt.title('Person 1')

    plt.subplot(1, 2, 2)
    plt.imshow(test_data_prepared[1][id])
    plt.title('Person 2')

    plt.show()